In [1]:
import MetaTrader5 as mt
import pandas as pd
import plotly.express as px
import matplotlib.pylab as plt
import numpy as np
import talib
from talipp.indicators import EMA, SMA, Stoch, DPO
from joblib import load
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, confusion_matrix, classification_report
from own_functions import *
import os
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute


mt.initialize()
login = 51708234
password ="4bM&wuVJcBTnjV"
server = "ICMarketsEU-Demo"
mt.login(login,password,server)

symbol = "GBPUSD"
timeframe = mt.TIMEFRAME_D1
ohlc_data = pd.DataFrame(mt.copy_rates_range(symbol, timeframe, datetime(2010, 1, 1), datetime(2024,7,6)))
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
df = ohlc_data[['time', 'open', 'high', 'low', 'close']].copy()

# Indicators
df['WILLR_15'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=15)
df['WILLR_23'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=23)
df['WILLR_42'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=42)
df['WILLR_145'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=145)

# Buy & Sell Flags
df['b_flag'] = 0
df['s_flag'] = 0

# Dropping NaN values and resetting index
df = df.dropna().reset_index(drop=True)

StopLoss = 1
TakeProfit = 2
BreakEvenRatio=StopLoss/(StopLoss+TakeProfit)
label_data(df,[StopLoss],[TakeProfit],80,symbol,False)

df.drop(columns=['b_flag'], inplace=True)

selected_signal_1 = 'WILLR_15'
df_melted_1 = df[['time', selected_signal_1]].copy()
df_melted_1["Symbols"] = symbol

df_rolled_1 = roll_time_series(df_melted_1, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X1 = extract_features(df_rolled_1.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_1, 
                      impute_function=impute, show_warnings=False)

X1 = X1.set_index(X1.index.map(lambda x: x[1]), drop=True)
X1.index.name = "time"
X1 = X1.dropna()

selected_signal_2 = 'WILLR_42'
df_melted_2 = df[['time', selected_signal_2]].copy()
df_melted_2["Symbols"] = symbol

df_rolled_2 = roll_time_series(df_melted_2, column_id="Symbols", column_sort="time",
                               max_timeshift=20, min_timeshift=5)

X2 = extract_features(df_rolled_2.drop("Symbols", axis=1), 
                      column_id="id", column_sort="time", column_value=selected_signal_2, 
                      impute_function=impute, show_warnings=False)

X2 = X2.set_index(X2.index.map(lambda x: x[1]), drop=True)
X2.index.name = "time"
X2 = X2.dropna()

X = pd.concat([X1, X2], axis=1, join='inner')
X = X.dropna()

# Align indices
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df = df[df.index.isin(X.index)]

X = pd.concat([X, df], axis=1, join='inner')

# Ensure b_flag is at the end after feature selection
X_df = select_features(X, X['s_flag'])
X_df = X_df[[col for col in X_df if col != 's_flag'] + ['s_flag']]

scaler = load('Dump_GBPUSD_D1_3112_Sell/scaler.joblib')
model = load('Dump_GBPUSD_D1_3112_Sell/model.joblib')

sum_fp = 0
sum_tp = 0

split = int(0.96 * len(X_df))

train_data, test_data = X_df.iloc[:split], X_df.iloc[split:]

# Train data
x_train = train_data.iloc[:, :-1].values
y_train = train_data['s_flag'].values
# Test data
x_test = test_data.iloc[:, :-1].values
y_test = test_data['s_flag'].values

x_test = scaler.transform(x_test)

y_pred = model.predict(x_test)


print("Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

false_positives = cm[0][1]
true_positives = cm[1][1]

sum_fp += false_positives
sum_tp += true_positives

precision = precision_score(y_test, y_pred)

print('WIN/LOSS-Diff:', round(100 * (precision - BreakEvenRatio), 2), '%')
print('sum_fp:', sum_fp)
print('sum_tp:', sum_tp)
print('precision:', precision)
print('Ratio total:', round(100 * (sum_tp / (sum_fp + sum_tp)), 2))
print('BreakEvenRatio:', round(BreakEvenRatio, 2))
print('____________________________________________________________________________________________________________________________')


Mean Candle: 0.011325125586530491


Feature Extraction: 100%|██████████| 30/30 [01:30<00:00,  3.01s/it]


Confusion Matrix:
[[135   1]
 [  9   0]]
WIN/LOSS-Diff: -33.33 %
sum_fp: 1
sum_tp: 0
precision: 0.0
Ratio total: 0.0
BreakEvenRatio: 0.33
____________________________________________________________________________________________________________________________
